In [1]:

import argparse
from copy import deepcopy as cpy

# from async_vi.MDP import *
import numpy as np
from bigmdp.data.dataset import SimpleReplayBuffer, PrioritizedReplayBuffer
from bigmdp.data.env_gym import SimpleNormalizeEnv
from bigmdp.mdp.MDP_GPU import FullMDP
from bigmdp.utils.image_wrappers import *
from bigmdp.utils.tmp_vi_helper import *
from bigmdp.utils.utils_directory import *
from bigmdp.utils.utils_log import *
from hyper_params import HYPERPARAMS

# Get all Arguments
parser = argparse.ArgumentParser()
parser.add_argument("--bottle_neck_size", help="size", type=int, default=32)
parser.add_argument("--discrete_bn", help="size", type=int, default=0)
parser.add_argument("--multiplyer", help="multiplyer of feature space", type=int, default=10)
parser.add_argument("--env", help="environment name", type=str, default="CartPole")
parser.add_argument("--name", help="Experiment name", type=str, default="CartPoleR1")
parser.add_argument("--load", help="Load the previous MDP ?", type=int, default=0)
parser.add_argument("--symbolic", help="Use Symbolic env if 1 else use image based env", type=int, default=1)
parser.add_argument("--steps_to_train", help="Number of steps to train the whole pipeline", type=int, default=0)
parser.add_argument("--rmax", help="Use rmax exploration?", type=int, default=0)
parser.add_argument("--strict_rmax", help="Use rmax exploration and rmax exploration alone?", type=int, default=0)
parser.add_argument("--video_every", help="get a rollout video every", type=int, default=999999999)
parser.add_argument("--backup_every", help="do a bellman backup every k frames", type=int, default=10)
parser.add_argument("--device", help="do backups on ?", type=str, default="GPU")
parser.add_argument("--save_transitions", help="do backups on ?", type=int, default=0)
parser.add_argument("--shaped_reward", help="Use shaped rewards?", type=int, default=0)
parser.add_argument("--load_time_string", help="timestep string", type=str, default="default_time")
parser.add_argument("--use_priority_buffer", help="Use priority buffer ?", type=int, default=0)
parser.add_argument("--render_every", help="render a episode every kth episode", type=int, default=0)
parser.add_argument("--internal_count_override", help="override internal count", type=int, default=0)


# args = parser.parse_args()
args = parser.parse_args("--env CartPole --multiplyer 5 --rmax 0 --backup_every 100 --render_every 10000".split(" "))

run_params = "_bn-" + str(args.bottle_neck_size) + \
             "_sym-" + str(args.symbolic) + \
             "_rmax-" + str(bool(args.rmax)) + \
             "_strict_rmax-" + str(bool(args.strict_rmax)) + \
             "_mult-" + str(args.multiplyer) + \
             "_bkp_f-" + str(args.backup_every) + \
             "_device-" + str(args.device) + \
             "_priority-" + str(args.use_priority_buffer)

base_file_path = "./result_dump/{}/{}".format(args.env, args.name + run_params)

create_hierarchy(base_file_path)

train_epochs = 20

log_dirs_dict, loggers_dict = get_advanced_log_dir_and_logger(ROOT_FOLDER="Symbolic" if args.symbolic else "Image",
                                                              EXP_ID=args.env + "-Dec", #todo remove for general
                                                              EXP_PARAMS=run_params,
                                                              load_time_string=args.load_time_string,
                                                              tb_log_keys=["tb_train_logger", "tb_valid_logger"])

if args.symbolic:
    params = HYPERPARAMS[args.env + "-sym"]
    if os.path.exists("./" + str(args.env) + "_env.pk"):
        print(" Environment  Loaded")
        env = torch.load("./" + str(args.env) + "_env.pk")
    else:
        env = SimpleNormalizeEnv(params["env_name"], max_episode_length=params["max_episode_length"])
        torch.save(env, "./" + str(args.env) + "_env.pk")
else:
    print("Not Implemented Yet")  # Todo
    params = HYPERPARAMS[args.env + "-img"]
    assert False

test_env = cpy(env)


K_s, K_i = params["replay_initial"], args.internal_count_override if args.internal_count_override else params[
    'internal_step_count_per_policy']
print(K_i, K_s)

if args.load:
    mdp = torch.load(base_file_path + "mdp_class.pth")
else:
    mdp = FullMDP(A=env.get_list_of_actions(),
                  ur=params["unknown_transition_reward"],
                  vi_params={"gamma": params["gamma"],
                             "slip_prob": params["slip_probability"],
                             "rmax_reward": params["rmax_reward"],
                             "rmax_thres": 2,
                             "balanced_explr": True,
                             "rmin": params["rmin"]},
                  policy_params={"unhash_array_len": env._env.observation_space.shape[0]},
                  MAX_S_COUNT=int(1e6),
                  weight_transitions=False,
                  default_mode = args.device)

all_rewards = []
eval_rewards, safe_eval_rewards = [], []
policy_fetch_time = []
bellman_backup_time = [9999]
train_buffer = PrioritizedReplayBuffer(int(5e5)) if args.use_priority_buffer else SimpleReplayBuffer(int(5e5))
valid_buffer = PrioritizedReplayBuffer(int(1e5)) if args.use_priority_buffer else SimpleReplayBuffer(int(5e5))

eps_tracker = EpsilonTracker(params)

frame_count = 0
warmup_eps = 10
eval_reward = 0

omit_list = ["end_state", "unknown_state"]

from bigmdp.data.dataset import gather_data_in_buffer
from learnt_mdp_helper import *

# Collect random Dataset # till replay initial\
random_policy = lambda s: env.sample_random_action()
train_buffer, info = gather_data_in_buffer(train_buffer, env, episodes=9999, render=False, policy=random_policy,
                                           frame_count=K_s, pad_attribute_fxn={"qval": lambda s: 0})
valid_buffer, info = gather_data_in_buffer(valid_buffer, env, episodes=9999, render=False, policy=random_policy,
                                           frame_count=int(K_s / 5), pad_attribute_fxn={"qval": lambda s: 0})

# recon_loss_wts = {h:0 for h in training_net.head_ids}
# recon_loss_wts["recon"] = 1
# training_net.default_loss_wts  = recon_loss_wts
# training_net.fit(train_buffer,valid_buffer, epochs=50)
frame_count = 0
print(frame_count)

from tqdm import tqdm





args.model_update_every = 500

outer_loop_frame_count = len(train_buffer)

    # torch.save((train_buffer.buffer, mdp, net.state_dict()), log_dirs_dict["py_log_dir"] + "/checkpoint" + ".pth")

 Environment  Loaded
20000 20000


2020-02-28 15:41:26,250:mylogger:Average Reward of collected trajectories:21.907
I0228 15:41:26.250660 140416558835520 dataset.py:292] Average Reward of collected trajectories:21.907
2020-02-28 15:41:26,353:mylogger:Average Reward of collected trajectories:23.585
I0228 15:41:26.353203 140416558835520 dataset.py:292] Average Reward of collected trajectories:23.585


Average Reward of collected trajectories:21.907
Average Reward of collected trajectories:23.585
0


In [2]:
params["evaluate_every"] = 5000
params["outer_iteration_count"] = 1
params["bellman_backup_every"] = 100
params["n_backups"] = 10
K_i = int(2e6)

In [3]:
def img_2_disc_fxn(s):
    if len(s)==1:
        return s[0] * args.multiplyer
    else:
        return [s_ * args.multiplyer for s_ in s]

In [4]:

outer_count = 0 
# Update the buffer with new values
# For Disc: Skipped

# Train your Network
# For Disc: Skipped

#Make MDP
mdp_T = FullMDP(A=env.get_list_of_actions(),
              ur=params["unknown_transition_reward"],
              vi_params={"gamma": params["gamma"],
                         "slip_prob": params["slip_probability"],
                         "rmax_reward": params["rmax_reward"],
                         "rmax_thres": 2,
                         "balanced_explr": True,
                         "rmin": params["rmin"]},
              policy_params={"unhash_array_len": env._env.observation_space.shape[0]},
              MAX_S_COUNT=int(1e6),
              weight_transitions=True,
              default_mode="GPU")

# Populate your MDP
mdp_T = populate_model_from_buffer(mdp_T, train_buffer, img_2_disc_fxn)
print("Size of MDP,", len(mdp_T.tD))

# Solve Your MDP
mdp_T.solve(eps=1, mode=args.device)

# for mdp_name, M in {"Tabular": mdp_T, "Tabular_Insertion": mdp_TI}.items():

opt_policy = lambda s: mdp_T.get_opt_action(hAsh(img_2_disc_fxn([s]).tolist()), mode = args.device)
safe_policy = lambda s: mdp_T.get_safe_action(hAsh(img_2_disc_fxn([s]).tolist()), mode = args.device)
explr_policy = lambda s: mdp_T.get_explr_action(hAsh(img_2_disc_fxn([s]).tolist()), mode = args.device)
random_policy = lambda s: env.sample_random_action()
eps_opt_policy = get_eps_policy(opt_policy, random_policy, epsilon=0.2)

if args.strict_rmax:
    explore_policies = {"explr_policy": explr_policy}
elif args.rmax:
    explore_policies = {"explr_policy": explr_policy, "opt_policy": opt_policy}
else:
    explore_policies = {"opt_policy": opt_policy, "eps_opt_policy":eps_opt_policy}

s = env.reset()
running_reward = 0
inner_loop_frame_count = 0
eps_count = 0

100%|██████████| 79/79 [00:05<00:00, 15.76it/s]


Size of MDP, 496
Time takedn to solve 2.43491792678833


In [ ]:
args.render_every = 10000

In [ ]:

for i in tqdm(range(int(1e8))):
    inner_loop_frame_count += 1
    outer_loop_frame_count += 1

    if args.render_every and eps_count % args.render_every == 0:
        env.render()

    if outer_loop_frame_count % params['bellman_backup_every'] == 0:
        st = time.time()
        mdp_T.do_optimal_backup(mode=args.device, n_backups=params["n_backups"])
        if args.rmax:
            mdp_T.do_explr_backup(mode=args.device, n_backups=params["n_backups"])
        bellman_backup_time.append(time.time() - st)

    policy_name, policy = "a", eps_opt_policy # list(explore_policies.items())[eps_count%len(explore_policies)]
    a = policy(s)

    ns, r, d, i = env.step(a)
    _d = False if d and i["max_episode_length_exceeded"] == True else d
    running_reward += r

    # add to buffer
    # Not necessary for Decimal Discretization bur sure
    exp = [s.tolist(), [a], ns.tolist(), [r], [_d]]
    train_buffer.add(exp, padded_info={"qval": 0})
    if np.random.randint(20) < 2:
        valid_buffer.add(exp, padded_info={"qval": 0})

    # Update MDP
    s_d, ns_d = img_2_disc_fxn([s]), img_2_disc_fxn([ns])
    hs_d, hns_d = hAsh(s_d.tolist()), hAsh(ns_d.tolist())
    mdp_T.consume_transition([hs_d, int(a), hns_d, float(r), int(_d)])
#     mdp_T.consume_transition((hAsh(hs.tolist()), int(a), hAsh(hns.tolist()), float(r), int(_d)))

    # prep for next loop
    s = ns

    # Housekeeping to omit while true loop
    if d:
        s = env.reset()
        running_reward = 0
        eps_count += 1

    if inner_loop_frame_count % params['evaluate_every'] == 0:
        mdp_T.do_optimal_backup(mode="GPU", n_backups=500)
        n_steps = outer_count * K_i + inner_loop_frame_count
        eval_reward = evaluate_on_env(test_env, opt_policy, eps_count=50, render=False)[0]
        
        col_header = ["Average Reward","MDP #States", "#MissingTrans/ #Trans","Missing %", "VI Error"] 
        meta_data = [10]*len(col_header) # max len of the data in the table to be printed (per column)
        print(' | '.join([s.ljust(max(meta_data[i], len(s)), '.') for i, s in enumerate(col_header)]))
        col_data = [eval_reward, 
                    len(mdp_T.tD),  
                    str(mdp_T.missing_state_action_count)+"/"+str(len(mdp_T.tD)*len(mdp_T.A)),
                    round(mdp_T.missing_state_action_count/mdp_T.total_state_action_count,4 ),
                    mdp_T.curr_vi_error       ]
        print(' | '.join([str(s).ljust(max(meta_data[i], len(col_header[i])),  " ") for i, s in enumerate(col_data)]))
        
        loggers_dict["tb_train_logger"].add_scalar("Optimal Avg Reward", n_steps)
        loggers_dict["tb_train_logger"].add_scalar("State Count", float(len(mdp_T.tD)), outer_loop_frame_count)
        loggers_dict["tb_train_logger"].add_scalar('Mising Transition Count', mdp.missing_state_action_count, outer_loop_frame_count)
        loggers_dict["tb_train_logger"].add_scalar("Opt VI Error",mdp_T.curr_vi_error, outer_loop_frame_count)
        loggers_dict["tb_train_logger"].add_scalar("Safe VI Error",mdp_T.s_curr_vi_error, outer_loop_frame_count)

    ####################################################

    # Not necessary for Decimal Experiments
    # if outer_loop_frame_count % args.checkpoint_every == 0:
    #     training_net._save_to_cache()
    #     torch.save((train_buffer.buffer, mdp, net.state_dict()),
    #                log_dirs_dict["py_log_dir"] + "/checkpoint" + ".pth")

# Not necessary for Decimal Experiments
# training_net._save_to_cache()

  0%|          | 5007/100000000 [00:27<3085:17:30,  9.00it/s]

Average Reward | MDP #States | #MissingTrans/ #Trans | Missing %. | VI Error..
78.84          | 657         | 343/1314              | 0.261      | 0.39984130859375


  0%|          | 10007/100000000 [00:55<2869:20:35,  9.68it/s]

Average Reward | MDP #States | #MissingTrans/ #Trans | Missing %. | VI Error..
116.4          | 854         | 495/1708              | 0.2898     | 0.593780517578125


  0%|          | 15006/100000000 [01:22<3054:40:43,  9.09it/s]

...

  0%|          | 260006/100000000 [24:56<3022:08:58,  9.17it/s]

Average Reward | MDP #States | #MissingTrans/ #Trans | Missing %. | VI Error..
299.36         | 3578        | 1959/7156             | 0.2738     | 0.940338134765625


  0%|          | 265006/100000000 [25:28<3945:51:20,  7.02it/s]

Average Reward | MDP #States | #MissingTrans/ #Trans | Missing %. | VI Error..
294.58         | 3611        | 1974/7222             | 0.2733     | 1.4024658203125


  0%|          | 267577/100000000 [25:38<99:31:30, 278.36it/s] 